# Capítulo 05 - Time Travel e Versionamento

## ⏳ Objetivo

Uma das features mais poderosas do Apache Iceberg é o **Time Travel** (viagem no tempo). Ele permite:
1.  Consultar os dados como eram no passado (por timestamp ou snapshot ID).
2.  Reverter a tabela para um estado anterior (Rollback).
3.  Reproduzir experimentos ou corrigir cargas erradas de dados.

Neste notebook, vamos:
1.  Gerar novas versões (snapshots) da tabela de vendas.
2.  Consultar versões antigas.
3.  Praticar o Rollback.

## 🔧 Requisitos

- Tabela `default.sales` criada no Capítulo 03.
- Biblioteca `pyiceberg` configurada.
- DuckDB para análise.

In [ ]:
import duckdb
import pandas as pd
import os
import time
from datetime import datetime
from pyiceberg.catalog.sql import SqlCatalog
import pyarrow as pa

# Configuração Visual
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# Configuração Paths
WAREHOUSE_PATH = './iceberg_warehouse'
CATALOG_DB = f"{WAREHOUSE_PATH}/catalog.db"

# Inicializar Catálogo
catalog = SqlCatalog(
    "local",
    **{
        "uri": f"sqlite:///{CATALOG_DB}",
        "warehouse": f"file://{os.path.abspath(WAREHOUSE_PATH)}",
    },
)

print("✅ Setup inicial concluído")
print(f"📂 Warehouse: {os.path.abspath(WAREHOUSE_PATH)}")

## 1. Preparar Dados: Criando Histórico

Para viajar no tempo, precisamos de um histórico. Vamos adicionar novos commits na tabela.

1.  **Snapshot 1**: Já criado no Cap 03 (Jan/Fev/Mar/Abr).
2.  **Snapshot 2**: Vamos adicionar vendas de **Maio**.
3.  **Snapshot 3**: Vamos adicionar vendas de **Junho** (com um erro proposital para corrigir depois).

In [ ]:
# Helper para adicionar dados
def append_sales_data(df_pandas, snapshot_name="Novo Snapshot"):
    try:
        tbl = catalog.load_table("default.sales")
        
        # Casting para compatibilidade (ns -> us)
        if 'order_date' in df_pandas.columns:
            df_pandas['order_date'] = df_pandas['order_date'].astype('datetime64[us]')
            
        arrow_table = pa.Table.from_pandas(df_pandas)
        tbl.append(arrow_table)
        print(f"✅ {snapshot_name}: {len(df_pandas)} linhas adicionadas.")
        print(f"   Snapshot ID: {tbl.current_snapshot().snapshot_id}")
    except Exception as e:
        print(f"❌ Erro ao adicionar dados: {e}")

# Criar Batch: Maio 2024
batch_may = pd.DataFrame({
    'order_id': range(1000, 1020),
    'customer_id': [f'CUST{i:03d}' for i in range(20)],
    'product_id': ['PROD005'] * 20,
    'order_date': pd.date_range('2024-05-01', periods=20, freq='1D'),
    'quantity': [1] * 20,
    'total_amount': [150] * 20
})

append_sales_data(batch_may, "Snapshot 2 (Maio)")

# Aguardar um pouco para diferencial de timestamp
time.sleep(1)

# Criar Batch: Junho 2024 (COM ERRO: valores duplicados/errados)
batch_june_error = pd.DataFrame({
    'order_id': range(2000, 2010),
    'customer_id': ['ERROR'] * 10,
    'product_id': ['INVALID'] * 10,
    'order_date': pd.date_range('2024-06-01', periods=10, freq='1D'),
    'quantity': [999] * 10,
    'total_amount': [999999] * 10
})

append_sales_data(batch_june_error, "Snapshot 3 (Junho - Errado)")

## 2. Inspecionar Histórico (Snapshots)

Vamos visualizar a lista de snapshots disponíveis usando a propriedade `metadata.snapshots` do PyIceberg.

In [ ]:
tbl = catalog.load_table("default.sales")
print(f"Versão Atual: {tbl.current_snapshot().snapshot_id}")

print("\n📜 Histórico de Snapshots:")
history = []
for s in tbl.metadata.snapshots:
    ts = datetime.fromtimestamp(s.timestamp_ms / 1000)
    history.append({
        'snapshot_id': s.snapshot_id,
        'timestamp': ts,
        'operation': s.summary.get('operation', 'unknown'),
        'records': s.summary.get('added-records', 0)
    })

df_history = pd.DataFrame(history)
print(df_history)

# Guardar IDs para uso posterior
snap_ids = df_history['snapshot_id'].tolist()
print(f"\nIDs capturados: {snap_ids}")

## 3. Time Travel: Consultando o Passado

Com os IDs de snapshot, podemos consultar como a tabela estava antes da inserção dos dados "errados" de Junho.

DuckDB nativo suporta `FOR VERSION AS OF`, mas devido a peculiaridades de caminhos no Windows, usaremos a abordagem segura:
`PyIceberg.scan(snapshot_id=...) -> Arrow -> DuckDB`

In [ ]:
def query_at_snapshot(snapshot_id, label):
    print(f"\n🔍 Consultando Snapshot: {snapshot_id} ({label})")
    try:
        # Load table at specific snapshot
        arrow_table = tbl.scan(snapshot_id=snapshot_id).to_arrow()
        
        # Analyze with DuckDB
        con = duckdb.connect()
        stats = con.execute(f"""
            SELECT
                count(*) as total_rows,
                sum(total_amount) as total_revenue,
                max(order_date) as last_order
            FROM arrow_table
        """).df()
        
        print(stats)
        return stats
    except Exception as e:
        print(f"❌ Erro: {e}")

# Último Snapshot (Com Erro)
query_at_snapshot(snap_ids[-1], "Atual - Com erro")

# Penúltimo Snapshot (Correto - Apenas Maio)
query_at_snapshot(snap_ids[-2], "Passado - Apenas Maio")

# Antepenúltimo (Se houver - Apenas carga inicial)
if len(snap_ids) >= 3:
    query_at_snapshot(snap_ids[-3], "Passado - Carga Inicial")

## 4. Rollback (Reversão)

Identificamos que o último snapshot contem dados errados (vendas de Junho com valores absurdos).
Vamos usar a API de transação do PyIceberg para reverter a tabela para o estado anterior (Rollback).

In [ ]:
# Identificar ID seguro (penúltimo)
safe_snapshot_id = snap_ids[-2]
print(f"🎯 Alvo de Restauração: {safe_snapshot_id}")

restore_branch = "restored_v1"

print(f"Criando branch '{restore_branch}'...")
try:
    with tbl.manage_snapshots() as ms:
        if restore_branch not in tbl.metadata.refs:
             print("Argument order try: snapshot_id, name")
             # Tentativa invertida conforme erro do Pydantic sugeriu
             ms.create_branch(safe_snapshot_id, restore_branch)
        else:
             print(f"ℹ️ Branch {restore_branch} já existe")
             
    print(f"✅ Branch check ok")
except Exception as e:
    print(f"⚠️ Erro ao gerenciar branch: {e}")

# Consultar dados usando a Branch
print(f"\nVerificando dados da branch '{restore_branch}':")
try:
    if restore_branch in tbl.metadata.refs:
        ref = tbl.metadata.refs[restore_branch]
        branch_snap_id = ref.snapshot_id
        print(f"Branch '{restore_branch}' aponta para Snapshot ID: {branch_snap_id}")
        query_at_snapshot(branch_snap_id, f"Branch: {restore_branch}")
    else:
        print(f"❌ Branch {restore_branch} não encontrada refs")
        
except Exception as e:
    print(f"❌ Falha leitura branch: {e}")

## 5. Time Travel por Timestamp

Além de ID, podemos viajar para um momento específico no tempo.
Vamos usar o `timestamp_ms` registrado no histórico.

In [ ]:
# Time Travel via Timestamp (Manual Resolution)
# Como PyIceberg scan() pode variar, a forma mais robusta é resolver o ID via metadados

target_time = df_history.iloc[0]['timestamp']
print(f"🕒 Buscando snapshot ativo em: {target_time}")

# Lógica: Encontrar o snapshot mais recente que seja <= target_time
# (No nosso caso, é exato, mas em produção seria <=)
snap_by_time = df_history[df_history['timestamp'] <= target_time].sort_values('timestamp', ascending=False).iloc[0]
target_id = snap_by_time['snapshot_id']

print(f"🎯 ID Resolvido: {target_id}")

try:
    # Scan usando o ID resolvido
    query_at_snapshot(target_id, f"Time Travel: {target_time}")
except Exception as e:
    print(f"❌ Erro Time Travel: {e}")

## ✅ Resumo

Neste capítulo:
1.  Geramos múltiplos Snapshots na tabela.
2.  Consultamos versões passadas (Time Travel) via ID.
3.  Realizamos um Rollback para corrigir dados errados.
4.  Aprendemos a usar timestamp para Time Travel.

**Próximo:** Capítulo 06 - Catálogos REST e Autenticação (ou manipulação avançada de partições).